# Temporal Convolutional Network

In [2]:
# !pip install darts


In [ ]:
# fix python path if working locally
from utils import fix_pythonpath_if_working_locally
fix_pythonpath_if_working_locally()

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from darts import TimeSeries
from darts.models import TCNModel, RNNModel
from darts.dataprocessing.transformers import Scaler
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.metrics import mape, r2_score
from darts.utils.missing_values import fill_missing_values
from darts.datasets import AirPassengersDataset, SunspotsDataset, EnergyDataset

ModuleNotFoundError: No module named 'darts'

!pip install darts


In [ ]:
# Read data:
ts = AirPassengersDataset().load()
scaler = Scaler()
ts = scaler.fit_transform(ts)  # scale the whole time series not caring about train/val split...

# We'll use the month as a covariate
month_series = datetime_attribute_timeseries(ts, attribute='month', one_hot=True)
scaler_month = Scaler()
month_series = scaler_month.fit_transform(month_series)

# Create training and validation sets:
train, val = ts.split_after(pd.Timestamp('19580801'))
train_month, val_month = month_series.split_after(pd.Timestamp('19580801'))

In [ ]:
model_air = TCNModel(
    input_chunk_length=13,
    output_chunk_length=12,
    n_epochs=500,
    dropout=0.1,
    dilation_base=2,
    weight_norm=True,
    kernel_size=5,
    num_filters=3,
    random_state=0
)

In [ ]:
model_air.fit(series=train,
              past_covariates=train_month,
              val_series=val,
              val_past_covariates=val_month,
              verbose=True)

In [ ]:
backtest = model_air.historical_forecasts(series=ts,
                                          past_covariates=month_series,
                                          start=0.7,
                                          forecast_horizon=6,
                                          retrain=False,
                                          verbose=True)

ts.plot(label='actual')
backtest.plot(label='backtest (H=6)')
plt.legend();